In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from analysis import *

In [47]:
SSI = pd.read_csv('../data/SSI.csv',index_col=0)[:162]
SSI.index = pd.to_datetime(SSI.index)


In [56]:
VIX = pd.read_excel("../data/vix.xlsx",index_col=0)[['VIX Index']]
VIX=VIX.reset_index(drop=False).resample('M', on='Date').sum()[1:163]
VIX

C:\Users\collo\AppData\Local\Temp\ipykernel_29616\3520573833.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  VIX=VIX.reset_index(drop=False).resample('M', on='Date').sum()[1:163]


,VIX Index
Date,
2006-07-31,306.52
2006-08-31,307.09
2006-09-30,243.69
2006-10-31,248.74
2006-11-30,227.17
...,...
2019-08-31,417.54
2019-09-30,311.18
2019-10-31,355.73


In [49]:
SP500 = pd.read_excel("../data/sp500.xlsx",index_col=0)[['PX_LAST']]
SP500.index = pd.to_datetime(SP500.index)
SP500=SP500.reset_index(drop=False).resample('M', on='Date').sum().pct_change()[6:169]

C:\Users\collo\AppData\Local\Temp\ipykernel_29616\180376769.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  SP500=SP500.reset_index(drop=False).resample('M', on='Date').sum().pct_change()[6:169]


In [50]:
pastor = pd.read_csv("../data/pastor_series.txt", delimiter='\t').reset_index(drop=False)
pastor.columns = ['Date','Agg_Liq','Innov_Liq','Traded_Liq']
pastor = pastor.set_index('Date')[['Agg_Liq']]
pastor.index = pd.to_datetime(pastor.index,format='%Y%m')
pastor=pastor.reset_index(drop=False).resample('M', on='Date').sum()

C:\Users\collo\AppData\Local\Temp\ipykernel_29616\559648182.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pastor=pastor.reset_index(drop=False).resample('M', on='Date').sum()


In [51]:
pastor

,Agg_Liq
Date,
1962-08-31,-0.017537
1962-09-30,-0.004075
1962-10-31,-0.104212
1962-11-30,-0.019742
1962-12-31,-0.005089
...,...
2022-08-31,0.015746
2022-09-30,-0.009563
2022-10-31,-0.023783


In [54]:
def control_analysis(serie,ssi,control):
    # regression des données entre elles avec un lag
    scaler = StandardScaler()
    ssi = pd.DataFrame(scaler.fit_transform(ssi),index=ssi.index,columns=ssi.columns)
    scaler1 = StandardScaler()
    control = pd.DataFrame(scaler1.fit_transform(control),index=control.index,columns=control.columns)
    x = pd.concat([ssi, control],axis=1).dropna()
    print(x)
    y = serie.shift(-1).dropna()
    x,y=index_aligned(x,y)
    results = get_OLS(x, y)
    return results,(x,y)

In [55]:
control_analysis(SP500,SSI,pastor)


                   0   Agg_Liq
2006-08-31 -5.753386  0.717677
2006-09-30 -2.312582 -0.531940
2006-10-31 -1.035161  0.970927
2006-11-30  0.026424  0.367496
2006-12-31 -0.490989  0.919115
...              ...       ...
2019-09-30  0.005584 -0.333764
2019-10-31  0.081019 -0.660482
2019-11-30 -0.250215 -0.298694
2019-12-31  0.207649  0.126035
2020-01-31 -0.223994 -0.236498

[162 rows x 2 columns]


(<statsmodels.regression.linear_model.RegressionResultsWrapper at 0x24072655f90>,
 (                   0   Agg_Liq
  2006-08-31 -5.753386  0.717677
  2006-09-30 -2.312582 -0.531940
  2006-10-31 -1.035161  0.970927
  2006-11-30  0.026424  0.367496
  2006-12-31 -0.490989  0.919115
  ...              ...       ...
  2019-08-31  0.001067  1.191096
  2019-09-30  0.005584 -0.333764
  2019-10-31  0.081019 -0.660482
  2019-11-30 -0.250215 -0.298694
  2019-12-31  0.207649  0.126035
  
  [161 rows x 2 columns],
               PX_LAST
  2006-08-31 -0.109726
  2006-09-30  0.138047
  2006-10-31 -0.027779
  2006-11-30 -0.028560
  2006-12-31  0.005466
  ...              ...
  2019-08-31 -0.064348
  2019-09-30  0.148272
  2019-10-31 -0.093280
  2019-11-30  0.074296
  2019-12-31  0.031936
  
  [161 rows x 1 columns]))

In [ ]:
# Données d'exemple
x = np.column_stack((SSI['0'], VIX['VIX Index']))
y = SP500['PX_LAST'].shift(-1).dropna()
x_with_const = sm.add_constant(x)

# Modélisation de la régression
model = sm.OLS(y, x_with_const)
results = model.fit()
print(results.summary())


In [ ]:
# Données d'exemple
x = np.column_stack((SSI['0'], pastor))
y = SP500['PX_LAST'].shift(-1).dropna()
x_with_const = sm.add_constant(x)

# Modélisation de la régression
model = sm.OLS(y, x_with_const)
results = model.fit()
print(results.summary())
